<a href="https://colab.research.google.com/github/capozzolicaluan/MSWEPtest/blob/main/extraiMSWEP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Limpar todas as variáveis
from IPython import get_ipython
get_ipython().magic('reset -sf')

# Código para reiniciar o ambiente do Colab
import os
os.kill(os.getpid(), 9)

In [1]:
pip install netcdf4

In [1]:
# Importando as bibliotecas necessárias
import requests
import xml.etree.ElementTree as ET
import pandas as pd
from datetime import datetime, timedelta
import gc
import os as os
from datetime import datetime, timedelta
from matplotlib import pyplot as plt
import seaborn as sns
from google.colab import data_table
import math
from matplotlib.colors import TABLEAU_COLORS, same_color
from netCDF4 import Dataset
import matplotlib.pyplot as plt
import glob
import xarray as xr
from tqdm import tqdm # para monitorar o progresso do processo


# Liberar memória manualmente
_ = gc.collect()

In [2]:
from google.colab import drive
drive.mount('/content/drive',force_remount=True)

Mounted at /content/drive


In [3]:
caminho_teste = "/content/drive/MyDrive/MSWEP_V280/NRT/3hourly"

In [4]:
lista_estacoes = pd.read_excel("/content/estacoes_mswep.xlsx", header = None, names=['codigo','lat','lon'])

In [5]:
arquivos=os.listdir(caminho_teste)

In [6]:
# Filtra apenas os arquivos que começam com os anos desejados
anos_validos = ("2023")
arquivos_filtrados = [arq for arq in arquivos if arq.startswith(anos_validos)]

In [16]:
lista_arquivos = [os.path.join(caminho_teste, nome) for nome in arquivos_filtrados]

In [13]:
lista_arquivos = lista_arquivos[0:100]

['/content/drive/MyDrive/MSWEP_V280/NRT/3hourly/2023276.21.nc',
 '/content/drive/MyDrive/MSWEP_V280/NRT/3hourly/2023277.00.nc',
 '/content/drive/MyDrive/MSWEP_V280/NRT/3hourly/2023277.03.nc',
 '/content/drive/MyDrive/MSWEP_V280/NRT/3hourly/2023277.06.nc',
 '/content/drive/MyDrive/MSWEP_V280/NRT/3hourly/2023277.09.nc',
 '/content/drive/MyDrive/MSWEP_V280/NRT/3hourly/2023277.12.nc',
 '/content/drive/MyDrive/MSWEP_V280/NRT/3hourly/2023277.15.nc',
 '/content/drive/MyDrive/MSWEP_V280/NRT/3hourly/2023277.18.nc']

In [1]:
teste

NameError: name 'teste' is not defined

In [17]:
lista_chuva = []

for arquivo in lista_arquivos:
    try:
        with xr.open_dataset(arquivo) as ds:
            # Lista para os valores de cada estação neste arquivo
            precipitation_at_point = [None] * len(lista_estacoes)

            for i in range(len(lista_estacoes)):
                lat = lista_estacoes.iloc[i]['lat']
                lon = lista_estacoes.iloc[i]['lon']

                # Seleciona valor de chuva mais próximo e converte para número
                valor = ds['precipitation'].sel(lat=lat, lon=lon, method='nearest').values.item()
                precipitation_at_point[i] = valor

            lista_chuva.append(precipitation_at_point)
            print(f"Processado: {arquivo}")

    except Exception as e:
        print(f"Erro ao processar {arquivo}: {e}")

    gc.collect()

Processado: /content/drive/MyDrive/MSWEP_V280/NRT/3hourly/2023276.21.nc
Processado: /content/drive/MyDrive/MSWEP_V280/NRT/3hourly/2023277.00.nc
Processado: /content/drive/MyDrive/MSWEP_V280/NRT/3hourly/2023277.03.nc
Processado: /content/drive/MyDrive/MSWEP_V280/NRT/3hourly/2023277.06.nc
Processado: /content/drive/MyDrive/MSWEP_V280/NRT/3hourly/2023277.09.nc
Processado: /content/drive/MyDrive/MSWEP_V280/NRT/3hourly/2023277.12.nc
Processado: /content/drive/MyDrive/MSWEP_V280/NRT/3hourly/2023277.15.nc
Processado: /content/drive/MyDrive/MSWEP_V280/NRT/3hourly/2023277.18.nc
Processado: /content/drive/MyDrive/MSWEP_V280/NRT/3hourly/2023277.21.nc
Processado: /content/drive/MyDrive/MSWEP_V280/NRT/3hourly/2023278.00.nc
Processado: /content/drive/MyDrive/MSWEP_V280/NRT/3hourly/2023278.03.nc
Processado: /content/drive/MyDrive/MSWEP_V280/NRT/3hourly/2023278.06.nc
Processado: /content/drive/MyDrive/MSWEP_V280/NRT/3hourly/2023278.09.nc
Processado: /content/drive/MyDrive/MSWEP_V280/NRT/3hourly/202327

KeyboardInterrupt: 

In [15]:
lista_chuva

[[0.008113183546811342,
  2.9513612389564514e-06,
  0.0019522026414051652,
  0.003627627156674862,
  0.004207318474072963,
  0.069832960201893,
  0.021074917749501765,
  0.02062783105066046,
  0.016223460901528597,
  0.00020036089699715376,
  0.4548367201932706,
  0.21880193694960326,
  0.1498095205752179,
  0.05492547666653991,
  0.12710088526364416,
  0.00019138911738991737,
  0.00139047356788069,
  0.0006748039741069078,
  0.0006055725971236825,
  0.21386618074029684,
  0.023013605969026685,
  0.02349643420893699,
  0.40952506655594334,
  0.071580212097615,
  0.011042207828722894,
  0.0008696841541677713,
  0.00020678172586485744],
 [0.004740791104268283,
  5.794980097562075e-06,
  0.0012709033908322453,
  0.00018201407510787249,
  0.00020819471683353186,
  0.014981942193116993,
  0.006165108876302838,
  0.0005814822507090867,
  0.00019149668514728546,
  0.0002044069697149098,
  0.20410681382054463,
  0.06529015285195783,
  0.06558307126397267,
  0.031041249400004745,
  0.0633350815

In [9]:
# Pasta de saída (cria se não existir)
pasta_saida = "/content/drive/MyDrive/Colab/txts"
os.makedirs(pasta_saida, exist_ok=True)

In [11]:
for arquivo in tqdm(lista_arquivos):
    try:
        with xr.open_dataset(arquivo) as ds:
            linhas = []

            for i in range(len(lista_estacoes)):
                est = lista_estacoes.iloc[i]
                lat, lon, nome = est['lat'], est['lon'], est['codigo']

                valor = ds['precipitation'].sel(lat=lat, lon=lon, method='nearest').values.item()
                linha = f"{nome},{lat},{lon},{valor:.4f}"
                linhas.append(linha)

        nome_base = os.path.basename(arquivo).replace(".nc", "")
        caminho_saida = os.path.join(pasta_saida, f"{nome_base}.txt")

        with open(caminho_saida, "w") as f:
            f.write("nome,lat,lon,chuva\n")
            f.write("\n".join(linhas))

    except Exception as e:
        print(f"Erro ao processar {arquivo}: {e}")

    gc.collect()

100%|██████████| 66/66 [00:21<00:00,  3.08it/s]
